## PyTorch Custom Dataset

### Importing a PyTorch and setting up device-agnostic code

In [ ]:
import torch
from torch import nn

In [ ]:
torch.__version__

'2.6.0+cu124'

In [ ]:
# Set up device-agnostic code
device = "cuda" if torch.cuda.is_available() else 'cpu'

In [ ]:
device

'cuda'

In [ ]:
!nvidia-smi

Mon Apr 14 14:38:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Pushing and pulling project into and from github
!git config --global user.name "Asliddin"
!git config --global user.email "asliddinmalikov999@gmail.com"


In [ ]:
!git clone https://github.com/tayfunai/PytorchPractise.git

Cloning into 'PytorchPractise'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 2), reused 4 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 936.80 KiB | 20.82 MiB/s, done.
Resolving deltas: 100% (2/2), done.


### 1. Get data
* Dataset I am going to work on is subset of Food101 dataset
* I am using just 3 classes of food out of 101 different categories in actual dataset
* My dataset is 10 percent of Food101 dataset images.

In [5]:
import requests
import zipfile
from pathlib import Path

# Setup path to a data folder
data_path = Path("data/")
image_path = data_path/"pizza_steak_sushi"

# This piece of code is to prevent redownloading dataset if it's already exist
if image_path.is_dir():
  print(f" {image_path} directory already exists... skipping download")
else:
  print(f"{image_path} does not exist, so creating a new one...")
  image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak and sushi data
with open(data_path/"pizza_steak_sushi.zip", "wb") as f:
  print(f"Downloading dataset from github...")
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  f.write(request.content)

# Unzip dataset
with zipfile.ZipFile(data_path/"pizza_steak_sushi.zip", "r") as zip_file:
  print(f"Unzipping dataset...")
  zip_file.extractall(image_path)

 data/pizza_steak_sushi directory already exists... skipping download
Unzipping dataset...


In [9]:
!cp -r /content/data /content/PytorchPractise

/root
